Get data

In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np
import altair as alt

## Countries

In [ ]:
! wget https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv

In [ ]:
by_country=pd.read_csv('time_series_19-covid-Confirmed.csv').groupby('Country/Region').sum()#.reset_index()

In [ ]:
d = by_country[by_country.columns[2:]].T#.reset_index()
deltas = (d-d.shift(1)).fillna(0)
deltas.index= pd.to_datetime(deltas.index)
deltas.index.name='date'
deltas = deltas.T.reset_index()
deltas.head()

In [ ]:
canonical = deltas.melt(value_vars=deltas.columns[1:], id_vars=['Country/Region'])
canonical.head()

In [ ]:
sorted_ = canonical.groupby('Country/Region').sum().sort_values('value', ascending=False).index.values
sorted_

In [ ]:
totals_area = alt.Chart(canonical, height=50).transform_aggregate(
    [alt.AggregatedFieldDef(op='sum',field='value', **{'as': 'area_total'})], groupby=['Country/Region']
).mark_rect().encode(
    alt.X('Country/Region:O', sort=list(sorted_)),
    alt.Y('area_total:Q', title='date'),
    color=alt.value('gray')
)
totals_area

In [ ]:
canonical['value2'] = np.maximum(canonical.value.fillna(0), 0)+0.1
canonical.columns = ['area', 'date', 'value', 'New cases']

In [ ]:
rect2 = alt.Chart(canonical).mark_rect().encode(
    alt.X('area:O', sort=list(sorted_)),
    alt.Y('monthdate(date):O', title='date'), 
    alt.Color('New cases:Q', scale=alt.Scale(type='log', scheme='lightmulti')),
)
rect2